In [13]:
import pandas as pd
import duckdb
import numpy as np

In [24]:
def process_data(file_path="../data/clean_movies.csv"):
    # Baca file CSV
    df = pd.read_csv(file_path)

    # Hilangkan spasi ekstra atau whitespace di sekitar kolom
    df.columns = df.columns.str.strip()

    # Normalisasi kolom Budget dan Income - hilangkan simbol $ dan tanda koma, lalu ubah menjadi float
    df['Budget'] = df['Budget'].replace({'\$': '', ',': ''}, regex=True).astype(float)
    df['Income'] = df['Income'].replace({'\$': '', ',': ''}, regex=True).astype(float)

    # Memisahkan beberapa nilai dalam kolom 'Directors' dan 'Stars' yang dipisahkan oleh koma menjadi list
    df['Directors'] = df['Directors'].str.split(', ')
    df['Stars'] = df['Stars'].str.split(', ')

    # Pastikan kolom 'Month' dan 'Year' dalam format yang benar
    df['Month'] = df['Month'].str.strip()
    df['Year'] = df['Year'].astype(int)

    # Memastikan tidak ada nilai null atau kosong yang bisa menyebabkan error di database
    df = df.fillna('Unknown')

    # Cetak contoh data setelah diproses untuk memastikan semuanya sudah benar
    print("Preview data setelah diproses:")
    print(df.head())

    return df

df = process_data()
df

ValueError: could not convert string to float: '"350000000 "'

In [29]:
df = pd.read_csv("../data/clean_movies.csv")
df.columns

Index(['Title', 'Rating', 'Year', 'Month', 'Certificate', 'Runtime',
       'Directors', 'Stars', 'Genre', 'Filming_location', 'Budget', 'Income',
       'Country_of_origin'],
      dtype='object')

In [42]:
import pandas as pd
import duckdb
import matplotlib.pyplot as plt

# Step 1: Read the CSV file
df = pd.read_csv('../data/clean_movies.csv')

# Step 2: Create DuckDB database and table
con = duckdb.connect('movies.duckdb')

# con.execute("""
# CREATE SEQUENCE seq_movieid START 1;
# """)

# Create the movies table
con.execute("""
    CREATE TABLE IF NOT EXISTS movies (
        id INTEGER DEFAULT nextval('seq_movieid'),
        title VARCHAR,
        rating FLOAT,
        year INTEGER,
        month VARCHAR,
        certificate VARCHAR,
        runtime INTEGER,
        directors VARCHAR,
        stars VARCHAR,
        genre VARCHAR,
        filming_location VARCHAR,
        budget FLOAT,
        income FLOAT,
        country_of_origin VARCHAR
    )
""")




# Insert data into the movies table
con.execute("""
    INSERT INTO movies 
        (
        title, rating, year, month, certificate, runtime, directors, stars,
        genre, filming_location, budget, income, country_of_origin
    ) SELECT * FROM df
""")

# # Step 3: Data Analysis

# # A. Grouping the data based on film certification
# print("Movies grouped by certification:")
# result = con.execute("""
#     SELECT certificate, COUNT(*) as count
#     FROM movies
#     GROUP BY certificate
#     ORDER BY count DESC
# """).fetchall()
# for row in result:
#     print(f"{row[0]}: {row[1]}")

# # B. Directors who have directed more than 1 film with rating above 7.5
# print("\nDirectors with more than one film rated above 7.5:")
# result = con.execute("""
#     WITH director_films AS (
#         SELECT unnest(string_split(directors, ', ')) as director, rating
#         FROM movies
#         WHERE rating > 7.5
#     )
#     SELECT director, COUNT(*) as film_count
#     FROM director_films
#     GROUP BY director
#     HAVING COUNT(*) > 1
#     ORDER BY film_count DESC
# """).fetchall()
# for row in result:
#     print(f"{row[0]}: {row[1]} films")

# # C. Films that made a profit
# print("\nProfitable films:")
# result = con.execute("""
#     SELECT title, (income - budget) as profit
#     FROM movies
#     WHERE income > budget
#     ORDER BY profit DESC
#     LIMIT 10
# """).fetchall()
# for row in result:
#     print(f"{row[0]}: ${row[1]:,.2f}")

# # D. Month with the largest number of films produced
# print("\nMonth with the most films produced:")
# result = con.execute("""
#     SELECT month, COUNT(*) as count
#     FROM movies
#     GROUP BY month
#     ORDER BY count DESC
#     LIMIT 1
# """).fetchone()
# print(f"{result[0]}: {result[1]} films")

# # E. Top 5 films with highest ratings per month
# print("\nTop 5 films with highest ratings per month:")
# result = con.execute("""
#     WITH ranked_movies AS (
#         SELECT month, title, rating,
#                ROW_NUMBER() OVER (PARTITION BY month ORDER BY rating DESC) as rank
#         FROM movies
#     )
#     SELECT month, title, rating
#     FROM ranked_movies
#     WHERE rank <= 5
#     ORDER BY month, rating
# """).fetchall()

# current_month = None
# for row in result:
#     if row[0] != current_month:
#         print(f"\n{row[0]}:")
#         current_month = row[0]
#     print(f"  {row[1]} (Rating: {row[2]})")

# # Visualization: Bar chart of movies by certification
# cert_data = pd.DataFrame(con.execute("SELECT certificate, COUNT(*) as count FROM movies GROUP BY certificate").fetchall(), 
#                          columns=['Certificate', 'Count'])
# plt.figure(figsize=(10, 6))
# plt.bar(cert_data['Certificate'], cert_data['Count'])
# plt.title('Number of Movies by Certification')
# plt.xlabel('Certification')
# plt.ylabel('Number of Movies')
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.savefig('movies_by_certification.png')
# plt.close()

# print("\nA bar chart of movies by certification has been saved as 'movies_by_certification.png'")

# # Close the connection
con.close()

ConversionException: Conversion Error: Could not convert string ' Where's My Donkey?!' to FLOAT

In [43]:
import pandas as pd

# Creating a DataFrame with Title and Genre, including titles with commas
data = {
    'Title': [
        'Inception',
        'The Godfather',
        'The Dark Knight',
        'Pulp Fiction',
        'The Shawshank Redemption',
        'Forrest Gump',
        'The Matrix',
        'Fight Club',
        'Interstellar',
        'Gladiator',
        'The Good, the Bad and the Ugly',  # Title with commas
        '12 Angry Men'  # Another title with a comma
    ],
    'Genre': [
        'Sci-Fi',
        'Crime',
        'Action',
        'Crime',
        'Drama',
        'Drama',
        'Sci-Fi',
        'Drama',
        'Sci-Fi',
        'Action',
        'Western',  # Genre for the title with commas
        'Drama'  # Genre for the title with commas
    ]
}

# Create the DataFrame
df = pd.DataFrame(data)

# Saving the DataFrame to a CSV file
csv_file_path = 'movies_with_commas.csv'  # Change this to your desired file path
df.to_csv(csv_file_path, index=False)  # No need for additional quoting as pandas does it automatically

print(f'DataFrame saved to {csv_file_path}')

DataFrame saved to movies_with_commas.csv


In [49]:
df2 = pd.read_csv('../data/clean_movies.csv')

# Step 1: Print the original row at index 844
print("Original row at index 844:")
print(df2.loc[844])  # or df2.iloc[844]

# Step 2: Edit the title in the row at index 844
df2.loc[844] = [
    "Nativity 3: Dude Where's My Donkey?!",  # Title
    3.6,                                     # Rating
    2014,                                    # Year
    "November",                              # Month
    "",                                      # Certificate (empty as per your example)
    109,                                     # Runtime
    "Debbie Isitt",                          # Directors
    '"Martin Clunes, Marc Wootton, Catherine Tate, Adam Garcia"',  # Stars
    '"Comedy, Family"',                      # Genre
    "UK",                                    # Filming Location
    "Unknown",                               # Budget
    '"$11,283,866 "',                        # Income
    "United Kingdom"                         # Country of Origin
]


# Step 3: Print the updated row to verify changes
print("\nUpdated row at index 844:")
print(df2.loc[844])

# Step 4: Save the changes to a new CSV file (optional)
df2.to_csv('../data/clean_movies_updated.csv', index=False)

Original row at index 844:
Title                                                          Nativity 3: Dude
Rating                                                      Where's My Donkey?!
Year                                                                        3.6
Month                                                                      2014
Certificate                                                            November
Runtime                                                                     109
Directors                                                          Debbie Isitt
Stars                "Martin Clunes, Marc Wootton, Catherine Tate, Adam Garcia"
Genre                                                          "Comedy, Family"
Filming_location                                                             UK
Budget                                                                  Unknown
Income                                                           "$11,283,866 "
Country_of_or

In [ ]:
import pandas as pd
df_clean = pd.read_csv('../data/clean_movies.csv')
# df_clean['Budget'] = df_clean['Budget'].replace('Unknown', np.nan)
# df_clean['Income'] = df_clean['Income'].replace('Unknown', np.nan)

# # Remove currency symbols, commas, quotes, and strip whitespace for Budget and Income
# for column in ['Budget', 'Income']:
#     df_clean[column] = df_clean[column].str.replace('$', '') \
#                                        .str.replace('DKK', '') \
#                                        .str.replace('€', '') \
#                                        .str.replace('₹', '') \
#                                        .str.replace('£', '') \
#                                        .str.replace('"', '') \
#                                        .str.replace(',', '') \
#                                        .str.strip()

#     # Convert both columns to numeric, with errors='coerce' to handle non-numeric values
#     df_clean[column] = pd.to_numeric(df_clean[column], errors='coerce')
pd.set_option('display.float_format', '{:.2f}'.format)

df_clean.iloc[0:111]

In [86]:
import duckdb
def connect_db(db_path='../db/movies.duckdb'):
    conn = duckdb.connect(db_path)
    return conn
conn = connect_db()
result = conn.execute('SELECT * FROM movie').fetchdf()
result
conn.close()






In [82]:
import duckdb
def connect_db(db_path='../db/movies.duckdb'):
    conn = duckdb.connect(db_path)
    return conn
conn = connect_db()
# result = conn.execute('SELECT * FROM movie').fetchdf()
result = conn.execute('SELECT * FROM directors').fetchdf()
# result = conn.execute('SELECT * FROM stars').fetchdf()
result
# conn.close()


,director_id,movie_id,director_name
0,1,1950,James Cameron
1,2,1951,Guillermo del Toro
2,3,1952,Mark Gustafson
3,4,1953,David Leitch
4,5,1954,Martin McDonagh
...,...,...,...
2136,2137,4086,Jee woon Kim
2137,2138,4087,Jan de Bont
2138,2139,4088,Mathieu Kassovitz
2139,2140,4089,Prachya Pinkaew
